## Testes com IEEE 13 Bus
_Autor: Leonardo Jaime_

### Imports

In [1]:
# !pip install py-dss-interface

# instalação da biblioteca do OpenDSS

# as outras bibliotecas estão inclusas no anaconda

In [2]:
# Importação das bibliotecas utilizadas
import py_dss_interface
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from bokeh.io import show, curdoc
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file
from bokeh.models import Range1d
from bokeh.themes import built_in_themes

Repósitorio GitHub: <a href="https://github.com/leojms/OpenDSS">Repositório</a> 

### Funções

##### Variáveis

In [3]:
# variaveis globais
dss = py_dss_interface.DSSDLL()
periodo = 24
kv = 4.16
kv2 = 0.48
bus = 680
horario = 0
linha_tensao = "line.650632"

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




##### Elementos

In [4]:
# Criação dos elementos

def medidor():
    # criando o objeto medidor via opendss, na linha de saida do transformador
    dss.text("new Energymeter.medidor line.650632 terminal=1")
    # adicionando um marcador em uma barra escolhida, apenas para demostrar como faz
    # dss.text("AddBusMarker bus=680 color=red size=6 code=15")


def carga(local):
    # criando a curva de carga, baseada na residencial
    # e editando todas as cargas, para que elas tenham esse padrao de comportamento
    dss.text("new loadshape.dia npts=24 interval=1")
    dss.text(fr"~ mult=(file=[{local}\curvas\curva_residencial.txt])")
    dss.text("batchedit load..* daily=dia")
    # curva alternativa
    # dss.text(r"~ mult=(file=[C:\Users\leonardo.simoes\OneDrive - Sistema FIEB\centro_comp\opendss\alimentadores\IEEE\13Bus\curvas\curva_res_red.txt])")


def fv1(local, kw = 75):
    # criação das curvas do sistema fotovoltaico
    # curva do comportamento do fator de temperatura de acordo com o aumento da temperatura
    dss.text("new XYcurve.ctemp npts=4 xarray=[0 25 75 100] yarray=[1.2 1.0 0.8 0.6]")
    # Curva de eficiência do inversor
    dss.text("new XYcurve.ceficiencia npts=5 xarray=[0.1 0.2 0.4 1 2] yarray=[0.86 0.9 0.93 0.97 0.8]")
    # Curva de irradiancia diaria baseada na irradiancia solar local
    dss.text("new loadshape.cirrad npts=24 interval=1")
    dss.text(fr"~ mult=(file=[{local}\curvas\curva_fv.txt])")
    # curva da variação de temperatura da placa fotovoltaica diaria
    dss.text("new tshape.t npts=24 interval=1")
    dss.text(fr"~ temp=(file=[{local}\curvas\curva_temp.txt])")
    # criação do sistema fotovoltaico
    n = 1
    # criação do objeto, associação das curvas criadas e criação dos monitores
    dss.text(f"new PVSystem.fv{n} phases=3 bus1={bus} kv={kv} irrad=0.98 pmpp={kw} kva=6000 temperature=25 pf=1")
    dss.text("~ %cutin=0.1 %cutout=0.1 effcurve=ceficiencia P-tCurve=ctemp Daily=cirrad Tdaily=t")
    dss.text(f"new monitor.fv_power{n} element=PVSystem.fv{n} terminal=1 mode=1 ppolar=no")
    dss.text(f"new monitor.fv_variaveis{n} element=PVSystem.fv{n} terminal=1 mode=3")

##### Cenários

In [5]:
def cenario_base(arquivo, horas, local):
    # cenario base, nenhum sistema externo criado conectado ao alimentador
    # função apenas resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot


def cenario_1(arquivo, horas, local):
    # cenario 1, sistema fotovoltaico fv1 conectado ao alimentador
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    fv1(local)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot


def cenario_2(arquivo, horas, local):
    # cenario 2, utilização do sistema fotovoltaico associado ao v2g
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    fv1(local, kw=500)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot


def cenario_3(arquivo, horas, local):
    # cenario 3, utilização do sistema fotovoltaico associado ao v2g
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    fv1(local, kw=2000)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    pot_base = dss.cktelement_powers()[0]
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot


def cenario_4(arquivo, horas, local):
    # cenario 3, utilização do sistema fotovoltaico associado ao v2g
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    fv1(local, kw=3000)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    #perc = abs(perdas / pot) * 100
    #dss.circuit_set_active_element(linha_tensao)
    #v = ((dss.cktelement_voltages_mag_ang()[0] + dss.cktelement_voltages_mag_ang()[2] + dss.cktelement_voltages_mag_ang()[4])/3)/((kv*1000)/math.sqrt(3))
    return perdas, pot


def cenario_5(arquivo, horas, local):
    # cenario 3, utilização do sistema fotovoltaico associado ao v2g
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor()
    carga(local)
    fv1(local, kw=5000)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha_tensao)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot

##### Exports

In [6]:
def excel(dataframe, nome_arquivo, local2):
    # função para exportar os dados dos cenarios para excel e gerar os dados para o dashboard
    nome = f'{nome_arquivo}.xlsx'
    dataframe.to_excel(fr'{local2}\cen_13_bus\{nome}')
    print(f"O arquivo de dados {nome} foi salvo em excel com sucesso")


def monitores_dss(arquivo, cenario, local, local2):
    # função para exportar os monitores do/dos sistema(s) fotovoltaico(s) de cada cenario
    # compilar arquivo
    dss.text(f"compile [{arquivo}]")
    # escolher cenario
    if cenario == 0:
        cenario_base(arquivo, horario, local)
    elif cenario == 1:
        cenario_1(arquivo, horario, local)
    elif cenario == 2:
        cenario_2(arquivo, horario, local)
    elif cenario == 3:
        cenario_3(arquivo, horario, local)
    elif cenario == 4:
        cenario_4(arquivo, horario, local)
    elif cenario == 5:
        cenario_5(arquivo, horario, local)
    else:
        print("O numero digitado para o cenario eh incorreto")
    # rodar simulação no regime determinado
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text(f"set number = {periodo}")
    # resolver o fator de potência
    dss.solution_solve()
    # condicional para plotar os monitores, de acordo com os monitores que forem desejados
    if cenario == 0:
        print("Nao ha monitores a serem mostrados")
    elif cenario>0 and cenario<=5:
        dss.monitors_write_name("fv_variaveis1")
        var1 = dss.monitors_channel(1)
        var2 = dss.monitors_channel(2)
        var3 = dss.monitors_channel(3)
        var4 = dss.monitors_channel(4)
        output_file(fr'{local2}\fv_13_bus.html')
        curdoc().theme = 'dark_minimal'
        irrad = figure(x_axis_label="Horas (h)", title="Irradiância (pu/m²)")
        irrad.x_range = Range1d(0, 24)
        irrad.line(x=range(1, len(var1)+1), y=var1, color='lime')
        pot = figure(x_axis_label="Horas (h)", title="Potência (kW)")
        pot.x_range = Range1d(0, 24)
        pot.line(x=range(1, len(var2)+1), y=var2, color='cyan')
        temp = figure(x_axis_label="Horas (h)", title="Fator de temperatura")
        temp.x_range = Range1d(0, 24)
        temp.line(x=range(1, len(var3)+1), y=var3, color='yellow')
        eff = figure(x_axis_label="Horas (h)", title="Eficiência")
        eff.x_range = Range1d(0, 24)
        eff.line(x=range(1, len(var4)+1), y=var4, color='hotpink')
        grid_layout = gridplot([[irrad, pot], [temp, eff]])
        show(grid_layout)
    else:
        print("O numero digitado para o cenario eh incorreto")

    
def plots_horario(arquivo, horario, cenario, local):
    # função para plotar e exportar dados referentes a algum horario especifico de algum cenario especifico
    # localização do diretorio de onde o opendss exporta as sobrecargas
    # caso seja modificado, criar outra variável com o novo endereço, que sera o endereço de onde está o arquivo do alimentador.dss
    # diretorio_senai = fr"{local}\IEEE13Nodeckt_EXP_OVERLOADS.CSV"

    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # condicional para escolher o cenario
    if cenario == 0:
        cenario_base(arquivo, horario, local)
    elif cenario == 1:
        cenario_1(arquivo, horario, local)
    elif cenario == 2:
        cenario_2(arquivo, horario, local)
    elif cenario == 3:
        cenario_3(arquivo, horario, local)
    elif cenario == 4:
        cenario_4(arquivo, horario, local)
    elif cenario == 5:
        cenario_5(arquivo, horario, local)
    else:
        print("O numero digitado para o cenario eh incorreto")
    # Plotar o circuito alimentador
    dss.text("plot circuit Power max=2000 y y C1=$00FF0000")
    # exportar em txt os elementos com sobrecarga
    # dss.text("Export overloads")
    # abrir o txt exportado acima, de acordo com o diretorio
    # dss.text(fr"fileedit [{diretorio_senai}]")
    

def plot_perdas(dados_base, dados1, dados2, dados3, dados4, dados5, label_base, label1, label2, label3, label4, label5, local2):
    # função para plotar os gráficos dos dados de perdas e potencia extraídos dos cenários
    # plots através da biblioteca bokeh, que apresenta gráficos interativos e exporta para html
    output_file(fr'{local2}\13_bus.html')
    curdoc().theme = 'dark_minimal'
    perdas = figure(x_axis_label="Horas (h)", title="Perdas (kW)")
    perdas.x_range = Range1d(0, 24)
    perdas.line(x=dados_base["horas (h)"], y=dados_base["perdas em kW"], color='white', legend_label=label_base)
    perdas.line(x=dados1["horas (h)"], y=dados1["perdas em kW"], color='hotpink', legend_label=label1)
    perdas.line(x=dados2["horas (h)"], y=dados2["perdas em kW"], color='deepskyblue', legend_label=label2)
    perdas.line(x=dados3["horas (h)"], y=dados3["perdas em kW"], color='yellow', legend_label=label3)
    perdas.line(x=dados4["horas (h)"], y=dados4["perdas em kW"], color='lawngreen', legend_label=label4)
    perdas.line(x=dados5["horas (h)"], y=dados5["perdas em kW"], color='aquamarine', legend_label=label5)
    potencia = figure(x_axis_label="Horas (h)", title="Potencia Total (kW)")
    potencia.x_range = Range1d(0, 24)
    potencia.line(x=dados_base["horas (h)"], y=dados_base["potencia em kW"], color='white', legend_label=label_base)
    potencia.line(x=dados1["horas (h)"], y=dados1["potencia em kW"], color='hotpink', legend_label=label1)
    potencia.line(x=dados2["horas (h)"], y=dados2["potencia em kW"], color='deepskyblue', legend_label=label2)
    potencia.line(x=dados3["horas (h)"], y=dados3["potencia em kW"], color='yellow', legend_label=label3)
    potencia.line(x=dados4["horas (h)"], y=dados4["potencia em kW"], color='lawngreen', legend_label=label4)
    potencia.line(x=dados5["horas (h)"], y=dados5["potencia em kW"], color='aquamarine', legend_label=label5)
    grid_layout = gridplot([[perdas, potencia]])
    show(grid_layout)

##### Processo

In [7]:
def listagem(lista_geral, perdas, potencia):
    # Organização dos dados de cada cenário em listas para cada variável
    perdas.append(lista_geral[0])
    potencia.append(lista_geral[1])


def banco_de_dados(dicio, dia, perdas, potencia):
    # Criação de um banco de dados com os dados que foram extraídos de cada cenário
    dicio = dict()
    dicio["horas (h)"] = dia
    dicio["perdas em kW"] = perdas
    dicio["potencia em kW"] = potencia
    dados = pd.DataFrame().from_dict(dicio)
    return dados


def organiza(arquivo_dss_original, local, local2):
    # função que chama as funções de processo criadas acima
    # criação do dia com as 24 horas
    dia = range(0, 24)

    # criação das listas
    lista_perdas_cen_base = []
    lista_pot_cen_base = []
    lista_perdas_cen_1 = []
    lista_pot_cen_1 = []
    lista_perdas_cen_2 = []
    lista_pot_cen_2 = []
    lista_perdas_cen_3 = []
    lista_pot_cen_3 = []
    lista_perdas_cen_4 = []
    lista_pot_cen_4 = []
    lista_perdas_cen_5 = []
    lista_pot_cen_5 = []

    # Organização dos dados nas listas referentes a cada variável
    for hora in dia:
        listagem(cenario_base(arquivo_dss_original, hora, local), lista_perdas_cen_base, lista_pot_cen_base)
        listagem(cenario_1(arquivo_dss_original, hora, local), lista_perdas_cen_1, lista_pot_cen_1)
        listagem(cenario_2(arquivo_dss_original, hora, local), lista_perdas_cen_2, lista_pot_cen_2)
        listagem(cenario_3(arquivo_dss_original, hora, local), lista_perdas_cen_3, lista_pot_cen_3)
        listagem(cenario_4(arquivo_dss_original, hora, local), lista_perdas_cen_4, lista_pot_cen_4)
        listagem(cenario_5(arquivo_dss_original, hora, local), lista_perdas_cen_5, lista_pot_cen_5)

    # criação dos dicionários
    dicio_base = dict()
    dicio1 = dict()
    dicio2 = dict()
    dicio3 = dict()
    dicio4 = dict()
    dicio5 = dict()
    # Retorno dos bancos de dados criados com os dados das listas pela função para uma variável local
    dados_base = banco_de_dados(dicio_base, dia, lista_perdas_cen_base, lista_pot_cen_base)
    dados1 = banco_de_dados(dicio1, dia, lista_perdas_cen_1, lista_pot_cen_1)
    dados2 = banco_de_dados(dicio2, dia, lista_perdas_cen_2, lista_pot_cen_2)
    dados3 = banco_de_dados(dicio3, dia, lista_perdas_cen_3, lista_pot_cen_3)
    dados4 = banco_de_dados(dicio4, dia, lista_perdas_cen_4, lista_pot_cen_4)
    dados5 = banco_de_dados(dicio5, dia, lista_perdas_cen_5, lista_pot_cen_5)


    # Plot dos cenários de perdas e potência
    plot_perdas(dados_base, dados1, dados2, dados3, dados4, dados5, "Cenario Base", "Cenario 1", "Cenario 2", "Cenario 3", "Cenario 4", "Cenario 5", local2)

    # exportação do banco de dados para excel
    excel(dados_base, "cenario base", local2)
    excel(dados1, "cenario 1", local2)
    excel(dados2, "cenario 2", local2)
    excel(dados3, "cenario 3", local2)
    excel(dados4, "cenario 4", local2)
    excel(dados5, "cenario 5", local2)
    # execução do plot
    


def processo(arquivo, hor, cen, local, local2):
    # Função para chamar a função acima e para chamar as outras funções de plot criadas
    organiza(arquivo, local, local2)
    plots_horario(arquivo, horario=hor, cenario=cen, local = local)
    #perfil_de_tensao(arquivo, cenario=cen, local=local)

### Calcular Fluxo

In [8]:
# usuario utilizado (esse método só funciona comigo mesmo kkkk)
# caso use em outro computador, apenas substituir o local e local2 por endereços da sua maquina
u_senai = "leonardo.simoes"
u_note = "leona"
# Diretório do arquivo do alimentador
# cas-o seja utilizado um outro alimentador, ou em outra máquina, é necessário criar a variável com o novo diretório
nome_arquivo = "IEEE13Nodeckt.dss"
local = fr"C:\Users\{u_senai}\OneDrive - Sistema FIEB\centro_comp\opendss\alimentadores\IEEE\13Bus"
local2 = fr"C:\Users\{u_senai}\OneDrive - Sistema FIEB\centro_comp\opendss\alimentadores"
arquivo = fr"{local}\{nome_arquivo}"

# caso seja executado em outra máquina, baixar os arquivos das curvas via github e criar variáveis para o novo endereço do arquivo

# função para executar os comandos de todas as funções criadas no código
processo(arquivo, cen=4, hor=12, local=local, local2=local2)
# pot_ve(arquivo_dss_original_senai, cen=1, hor=19)

#############################
# IMPORTANTE

# O endereço local 2 refere-se a pasta anterior, onde serão exportados o gráfico de perdas em html e o arquivo excel
# com os dados dos cenários
# O local se refere a onde fica o arquivo do circuito e a pasta com os arquivos das curvas
# Salvar a pasta com os arquivos do circuito que fica na pasta do opendss em outro lugar, de maneira a evitar erros de adm
#############################

O arquivo de dados cenario base.xlsx foi salvo em excel com sucesso
O arquivo de dados cenario 1.xlsx foi salvo em excel com sucesso
O arquivo de dados cenario 2.xlsx foi salvo em excel com sucesso
O arquivo de dados cenario 3.xlsx foi salvo em excel com sucesso
O arquivo de dados cenario 4.xlsx foi salvo em excel com sucesso
O arquivo de dados cenario 5.xlsx foi salvo em excel com sucesso


### Monitores OpenDSS

In [9]:
# Função para gerar os monitores

monitores_dss(arquivo, cenario=2, local=local, local2=local2)

### Dados FV

In [10]:
dss.text(f"compile [{arquivo}]")

dia = range(0, 24)
d = dict()

for cenario in range(1, 6):
    if cenario == 1:
        cenario_1(arquivo, horario, local)
    elif cenario == 2:
        cenario_2(arquivo, horario, local)
    elif cenario == 3:
        cenario_3(arquivo, horario, local)
    elif cenario == 4:
        cenario_4(arquivo, horario, local)
    elif cenario == 5:
        cenario_5(arquivo, horario, local)
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text(f"set number = {periodo}")
    dss.solution_solve()
    dss.monitors_write_name("fv_variaveis1")
    var1 = dss.monitors_channel(1)
    var2 = dss.monitors_channel(2)
    var3 = dss.monitors_channel(3)
    var4 = dss.monitors_channel(4)
    d[f"Irradiância Cenário {cenario} (pu/m²)"] = var1
    d[f"Potência Cenário {cenario} (kW)"] = var2
    d[f"Fator de temperatura Cenário {cenario}"] = var3
    d[f"Eficiência Cenário {cenario}"] = var4
    
d[f"Irradiância Cenário Base (pu/m²)"] = np.zeros(24)
d[f"Potência Cenário Base (kW)"] = np.zeros(24)
d[f"Fator de temperatura Cenário Base"] = np.zeros(24)
d[f"Eficiência Cenário Base"] = np.zeros(24)     
df = pd.DataFrame().from_dict(d)
nome = f'Variáveis FV.xlsx'
df.to_excel(fr'{local2}\cen_13_bus\{nome}')
print(f"O arquivo de dados {nome} foi salvo em excel com sucesso")

        

O arquivo de dados Variáveis FV.xlsx foi salvo em excel com sucesso
